In [1]:
import momepy as mm
import geopandas as gpd

In [2]:
path = 'files/AMS/elements.gpkg'

In [3]:
tess = gpd.read_file(path, layer="tessellation")
blg = gpd.read_file(path, layer="buildings")
streets = gpd.read_file(path, layer="network")

In [4]:
blg['nID'] = mm.get_network_id(blg, streets, 'nID', min_size=300)
tess = tess.merge(blg[['uID', 'nID']], on='uID', how='left')

Generating centroids...
Generating rtree...


Snapping: 100%|██████████| 252385/252385 [17:10<00:00, 244.84it/s]
/home/arch_ubuntu/Documents/mf/momepy/momepy/elements.py:731: UserWarning: Some objects were not attached to the network. Set larger min_size. 71 affected elements
  "Set larger min_size. {} affected elements".format(sum(series.isnull()))


In [9]:
tess.to_file(path, layer="tessellation", driver="GPKG")
blg.to_file(path, layer="buildings", driver="GPKG")

In [4]:
streets["sdsLen"] = mm.Perimeter(streets).series
tess["stcSAl"] = mm.StreetAlignment(tess, streets, "stcOri", "nID").series
blg["stbSAl"] = mm.StreetAlignment(blg, streets, "stbOri", "nID").series

profile = mm.StreetProfile(streets, blg, heights='sdbHei', distance=3)
streets["sdsSPW"] = profile.w
streets["sdsSPH"] = profile.h
streets["sdsSPR"] = profile.p
streets["sdsSPO"] = profile.o
streets["sdsSWD"] = profile.wd
streets["sdsSHD"] = profile.hd
 
streets["sssLin"] = mm.Linearity(streets).series
streets["sdsAre"] = mm.Reached(streets, tess, "nID", "nID", mode="sum", values="sdcAre").series
streets["sisBpM"] = mm.Count(streets, blg, "nID", "nID", weighted=True).series

100%|██████████| 38963/38963 [02:01<00:00, 320.64it/s]


In [5]:
tess.to_file(path, layer="tessellation", driver="GPKG")
blg.to_file(path, layer="buildings", driver="GPKG")
streets.to_file(path, layer="network", driver="GPKG")

In [7]:
import libpysal

str_q1 = libpysal.weights.contiguity.Queen.from_dataframe(streets)
 
streets["misRea"] = mm.Reached(
    streets, tess, "nID", "nID", spatial_weights=str_q1, mode="count"
).series
streets["mdsAre"] = mm.Reached(streets, tess, "nID", "nID", spatial_weights=str_q1,
                               mode="sum").series
 

/home/arch_ubuntu/miniconda/envs/momepy_env/lib/python3.7/site-packages/libpysal/weights/weights.py:165: UserWarning: The weights matrix is not fully connected: 
 There are 33 disconnected components.
 There are 24 islands with ids: 4947, 9743, 11845, 11847, 12236, 13201, 13384, 17845, 19228, 20095, 22403, 23066, 24566, 27980, 28348, 28569, 34489, 34499, 34517, 34774, 34961, 35169, 35609, 38739.
  warnings.warn(message)
100%|██████████| 38963/38963 [03:51<00:00, 168.38it/s]


In [8]:
graph = mm.gdf_to_nx(streets)
 
print("node degree")
graph = mm.node_degree(graph)
 
print("subgraph")
graph = mm.subgraph(
    graph,
    radius=5,
    meshedness=True,
    cds_length=False,
    mode="sum",
    degree="degree",
    length="mm_len",
    mean_node_degree=False,
    proportion={0: True, 3: True, 4: True},
    cyclomatic=False,
    edge_node_ratio=False,
    gamma=False,
    local_closeness=True,
    closeness_weight="mm_len",
)
print("cds length")
graph = mm.cds_length(graph, radius=3, name="ldsCDL")
 
print("clustering")
graph = mm.clustering(graph, name="xcnSCl")
 
print("mean_node_dist")
graph = mm.mean_node_dist(graph, name="mtdMDi")
 
nodes, edges, sw = mm.nx_to_gdf(graph, spatial_weights=True)
 
print("saving")
nodes.to_file(path, layer="nodes", driver="GPKG")
edges.to_file(path, layer="edges", driver="GPKG")
 
fo = libpysal.io.open("files/AMS/AMSnodes.gal", "w")
fo.write(sw)
fo.close()
 
edges_w3 = mm.sw_high(k=3, gdf=edges)
edges["ldsMSL"] = mm.SegmentsLength(edges, spatial_weights=edges_w3, mean=True).series
 
edges["ldsRea"] = mm.Reached(edges, tess, "nID", "nID", spatial_weights=edges_w3).series
edges["ldsRea"] = mm.Reached(
    edges, tess, "nID", "nID", spatial_weights=edges_w3, mode="sum", values="sdcAre"
).series
 
nodes_w5 = mm.sw_high(k=5, weights=sw)
nodes["lddNDe"] = mm.NodeDensity(nodes, edges, nodes_w5).series
nodes["linWID"] = mm.NodeDensity(
    nodes, edges, nodes_w5, weighted=True, node_degree="degree"
).series
 
blg["nodeID"] = mm.get_node_id(blg, nodes, edges, "nodeID", "nID")
tess = tess.merge(blg[["uID", "nodeID"]], on="uID", how="left")
 
nodes_w3 = mm.sw_high(k=3, weights=sw)
 
nodes["lddRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", nodes_w3).series
nodes["lddARe"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", nodes_w3, mode="sum", values="sdcAre"
).series
 
nodes["sddAre"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", mode="sum", values="sdcAre"
).series
nodes["midRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", spatial_weights=sw).series
nodes["midAre"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", spatial_weights=sw, mode="sum", values="sdcAre"
).series
 
nodes.rename(
    columns={
        "degree": "mtdDeg",
        "meshedness": "lcdMes",
        "local_closeness": "lcnClo",
        "proportion_3": "linP3W",
        "proportion_4": "linP4W",
        "proportion_0": "linPDE",
    }, inplace=True
)
 
print("saving")
nodes.to_file(path, layer="nodes", driver="GPKG")
edges.to_file(path, layer="edges", driver="GPKG")
tess.to_file(path, layer="tessellation", driver="GPKG")
blg.to_file(path, layer="buildings", driver="GPKG")

node degree
subgraph


100%|██████████| 28185/28185 [01:30<00:00, 311.31it/s]


cds length


100%|██████████| 28185/28185 [00:31<00:00, 897.76it/s] 


clustering
mean_node_dist


100%|██████████| 28185/28185 [00:01<00:00, 25632.28it/s]
/home/arch_ubuntu/miniconda/envs/momepy_env/lib/python3.7/site-packages/libpysal/weights/weights.py:165: UserWarning: The weights matrix is not fully connected: 
 There are 41 disconnected components.
  warnings.warn(message)


saving


100%|██████████| 28185/28185 [02:22<00:00, 197.33it/s]


saving
